I'll refer to Gnedin et al. (2014) as "G14" hereafter.

In [ ]:
import os
import math

# Third-party
from astropy.visualization import hist
from astropy.constants import G
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.integrate import odeint, quad
from scipy.stats import powerlaw, rv_continuous
from scipy.optimize import root, minimize
from scipy.special import gamma, gammainc
from scipy.interpolate import InterpolatedUnivariateSpline
import h5py

# Custom
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

In [ ]:
seed = 42 # for random numbers

In [ ]:
m_tot = 5E10 # Msun
n_s = 2.2
r_s = 4. # kpc
z = 3. # formation epoch

### Cumulative mass for Sersic profile

In [ ]:
# Adapted from Oleg's C code (uncluster/src/gc_evolution.c)
# for MW, the parameters (r_s, n_s, etc.) don't evolve

def ell_model_Menc_grid(m_tot, n_s, r_s, n_grid=501):
    rmax = 99. # kpc - maximum radius
    an = 2 * n_s
    bn = 2. * n_s - 1./3. + 0.0098765/n_s + 0.0018/n_s**2
    
    argc = bn * (rmax/r_s) ** (1./n_s)
    gamnorm = gammainc(an, argc)
    
    r_ser = np.zeros(n_grid)
    m_ser = np.zeros(n_grid)
    for i in range(n_grid):
        r_ser[i] = r = 10**(-3. + i/100.)
        arg = bn * (r/r_s)**(1./n_s)
        m_ser[i] = gammainc(an, arg) / gamnorm
    
    return r_ser, m_ser

_rgrid, _mgrid = ell_model_Menc_grid(m_tot, n_s, r_s)

plt.figure(figsize=(6,6))
plt.loglog(_r, _m * m_tot * 0.012)
plt.xlim(1E-3, 1E2)
plt.ylim(1E6, 1E9)
plt.title("Compare to Fig. 3 in G14", fontsize=16)

In [ ]:
def sample_radii(size=1):
    # Sample radii by inverting cumulative mass function (using interpolation)
    interp_func = interp1d(_mgrid, _rgrid)
    return interp_func(np.random.uniform(0, 1, size=size))

def mass_enc_stars(r):
    interp_func = InterpolatedUnivariateSpline(_rgrid, _mgrid * m_tot, k=1)
    return interp_func(r)

### Circular velocity curve

In [ ]:
v_c = np.sqrt(G * 1E12*u.Msun / (20*u.kpc) * (np.log(2.)-0.5))
v_c = galactic.decompose(v_c)
print(v_c)
halo = gp.SphericalNFWPotential(v_c=v_c, r_s=20*u.kpc, units=galactic) # M(<250kpc) = 10^12 Msun

In [ ]:
def circ_vel(r):
    q = np.zeros((3,r.size))*r.unit
    q[0] = r
    dPhi_dr = halo.gradient(q)[0]
    return np.sqrt(r*np.abs(dPhi_dr) + G * mass_enc_stars(r)*u.Msun / r).to(u.km/u.s)

In [ ]:
pl.plot(r, circ_vel(r*u.kpc), marker=None)
pl.xlabel('$r$ [kpc]')
pl.ylabel(r'$v_{\rm c}(r)$ [km ${\rm s}^{-1}$]')

---

In [ ]:
N_gcs = 8000
t_evolve = 11.5*u.Gyr

## Sample masses from power law

In [ ]:
def sample_masses(M_min, M_max, size=1):
    M_min = M_min.to(u.Msun).value
    M_max = M_max.to(u.Msun).value
    
    class MassFunction(rv_continuous):
        def _pdf(self, x):
            return x**-2 / (1/self.a - 1/self.b)
        
    return MassFunction(a=M_min, b=M_max).rvs(size=size) * u.Msun

In [ ]:
np.random.seed(seed)
gc_masses = sample_masses(M_min=1E4*u.Msun, M_max=1E7*u.Msun, size=N_gcs)
gc_r = sample_radii(size=N_gcs) * u.kpc

Compare with Figure 2 in G14

In [ ]:
fig,axes = pl.subplots(1,2,figsize=(12,6))
axes[0].hist(gc_masses, bins=np.logspace(4,7.1,9));
axes[0].set_xscale('log')
axes[0].set_yscale('log')
axes[0].set_xlim(1E3, 3E7)
axes[0].set_ylim(5E-1, 1E4)
axes[0].set_xlabel(r"Mass [${\rm M}_\odot$]")
axes[0].set_ylabel(r"$N$")

bins = np.logspace(-1.,2.,16)
H,_ = np.histogram(gc_r, bins=bins)

V = 4/3*np.pi*(bins[1:]**3 - bins[:-1]**3)
bin_cen = (bins[1:]+bins[:-1])/2.
axes[1].plot(bin_cen, H/V, ls='--', marker=None)

axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].set_xlim(1E-1, 1E2)
axes[1].set_ylim(1E-7, 1E2)
axes[1].set_xlabel(r"$r$ [kpc]")
axes[1].set_ylabel('GC density [kpc$^{-3}$]')

fig.tight_layout()

## Compute disruption times

In [ ]:
# def P(r, pot):
#     return 41.4 * (r.to(u.kpc).value) / (circ_vel(r, pot).to(u.km/u.s).value)
    
# def t_tid(r, M, pot, α=2/3.):
#     """ Tidal disruption timescale """
#     return 10*u.Gyr * (M / (2E5*u.Msun))**α * P(r, pot)

# def t_iso(M):
#     """ Isolation disruption timescale (2-body evaporation)"""
#     return 17*u.Gyr * (M / (2E5*u.Msun))

# def t_df(r, M, pot, f_e=0.5):
#     return 0.45*u.Gyr * (r.to(u.kpc).value)**2 * (circ_vel(r, pot).to(u.km/u.s).value) * (M/(1E5*u.Msun))**-1 * f_e

# def F(y, t):
#     M,r2 = y
    
#     r = np.sqrt(r2)
#     min_t = np.min([t_tid(r*u.kpc, M*u.Msun, pot).to(u.Gyr).value, 
#                     t_iso(M*u.Msun).to(u.Gyr).value], axis=0)
    
#     M_dot = -M / min_t
#     r2_dot = -r2 / t_df(r*u.kpc, M*u.Msun, pot).to(u.Gyr).value
    
#     return [float(M_dot), float(r2_dot)]

In [ ]:
_G = G.decompose(galactic).value

In [ ]:
_fac = (1*u.kpc/u.Myr).to(u.km/u.s).value
def _circ_vel_kms(r):
    r = float(r)
    dPhi_dr = halo._gradient(np.array([r,0.,0.]))[0]
    return np.sqrt(np.abs(r*dPhi_dr) + _G*mass_enc_stars(r)/r) * _fac

def P(r):
    return 41.4 * r / _circ_vel_kms(r)
    
def t_tid(r, M, α=2/3.):
    """ Tidal disruption timescale """
    return 10. * (M / 2E5)**α * P(r)

def t_iso(M):
    """ Isolation disruption timescale (2-body evaporation)"""
    return 17. * (M / 2E5)

def t_df(r, M, f_e=0.5):
    return 0.45 * r*r * _circ_vel_kms(r) * (M/1E5)**-1 * f_e

def F(y, t):
    M,r2 = y
    
    r = np.sqrt(r2)
    min_t = np.min([t_tid(r, M), 
                    t_iso(M)], axis=0)
    
    M_dot = -M/min_t
    r2_dot = -r2/t_df(r, M)
    
    return [float(M_dot), float(r2_dot)]

In [ ]:
fname = "../output/gc-evolve.h5"
if not os.path.exists(fname) or True:
    t_grid = np.linspace(0., t_evolve.value, 4096)

    with h5py.File(fname, "w") as f:
        f.create_dataset('time', data=t_grid)
        f.create_group('mass')
        f.create_group('radius')

        f['time'].attrs['unit'] = 'Gyr'
        f['mass'].attrs['unit'] = 'solMass'
        f['radius'].attrs['unit'] = 'kpc'

        for i,_r,_M in zip(range(N_gcs), gc_r.to(u.kpc).value, gc_masses.to(u.Msun).value):
            if (i % 1000) == 0:
                print(i)

            M_r2 = odeint(F, [_M, _r**2], t=t_grid)
            f['mass'].create_dataset(str(i), data=M_r2[:,0])
            f['radius'].create_dataset(str(i), data=np.sqrt(M_r2[:,1]))

In [ ]:
t_disrupt = np.zeros(N_gcs)
final_r = np.zeros(N_gcs)
final_m = np.zeros(N_gcs)
did_disrupt = np.zeros(N_gcs).astype(bool)
with h5py.File("../output/gc-evolve.h5", "r") as f:
    for i in range(N_gcs):
        _mass = f['mass/{}'.format(i)][:]
        _radius = f['radius/{}'.format(i)][:]
        
        if np.any(np.isnan(_mass)) or np.any(_mass < 0.):
            t_disrupt[i] = f['time'][np.isfinite(_mass) & (_mass >= 0)][-1]
            final_r[i] = _radius[np.isfinite(_mass) & (_mass >= 0)][-1]
            final_m[i] = 0.
            did_disrupt[i] = True
            
        else:
            t_disrupt[i] = np.nan
            final_r[i] = _radius[-1]
            final_m[i] = _mass[-1]
            did_disrupt[i] = False
            
    t_disrupt = t_disrupt*u.Unit(f['time'].attrs['unit'])
    final_r = final_r*u.Unit(f['radius'].attrs['unit'])

In [ ]:
print("{} did not disrupt".format(N_gcs-did_disrupt.sum()))
print("{:.0%}".format(did_disrupt.sum() / N_gcs))

In [ ]:
pl.figure(figsize=(6,5))
hist((t_disrupt - t_evolve)[did_disrupt], bins='scott')
pl.yscale('log')
pl.xlabel('Disruption time [Gyr]')
pl.ylabel('$N$')
pl.tight_layout()
# pl.savefig("/Users/adrian/projects/how-many-streams/plots/t_disrupt.pdf")

TODO: now i have disruption times, masses, radii, and mass-loss rates I can run simulations with some eccentricity distribution

---

In [ ]:
fig,axes = pl.subplots(1,2,figsize=(12,6))

axes[0].scatter(gc_masses, final_m, c=np.log10(gc_r.value), cmap='magma_r', alpha=0.75, s=4)
axes[0].set_xscale('log')
axes[0].set_yscale('log')
axes[0].set_xlim(1E2,1E7)
axes[0].set_ylim(1E2,1E7)
axes[0].set_xlabel('$M_i$ [kpc]')
axes[0].set_ylabel('$M_f$ [kpc]')

axes[1].scatter(gc_r, final_r, c=np.log10(gc_masses.value), cmap='magma_r', alpha=0.75, s=4)
axes[1].set_xlim(0,50)
axes[1].set_ylim(0,50)
axes[1].set_xlabel('$r_i$ [kpc]')
axes[1].set_ylabel('$r_f$ [kpc]')

In [ ]:
from astropy.io import fits
import astropy.coordinates as coord

In [ ]:
harris_tbl = fits.getdata("../data/harris-gc-catalog.fits", 1)

In [ ]:
harris_c = coord.SkyCoord(ra=harris_tbl['RA']*u.degree, dec=harris_tbl['DEC']*u.degree,
                          distance=harris_tbl['HELIO_DISTANCE']*u.kpc)
harris_gc_dist = harris_c.transform_to(coord.Galactocentric).represent_as(coord.PhysicsSphericalRepresentation).r

In [ ]:
fig,axes = pl.subplots(1,2,figsize=(12,6))

axes[0].hist(gc_masses, bins=np.logspace(4,7.1,9));
axes[0].hist(final_m[~did_disrupt], bins=np.logspace(3,7.1,12));

axes[0].set_xscale('log')
axes[0].set_yscale('log')
axes[0].set_xlim(1E3, 3E7)
axes[0].set_ylim(5E-1, 1E4)
axes[0].set_xlabel(r"Mass [${\rm M}_\odot$]")
axes[0].set_ylabel(r"$N$")

bins = np.logspace(-1.,2.,16)
H,_ = np.histogram(gc_r, bins=bins)
data_H,_ = np.histogram(harris_gc_dist, bins=bins)

V = 4/3*np.pi*(bins[1:]**3 - bins[:-1]**3)
bin_cen = (bins[1:]+bins[:-1])/2.
axes[1].plot(bin_cen, H/V, ls='--', marker=None)
axes[1].errorbar(bin_cen, data_H/V, np.sqrt(data_H)/V, 
                 color='k', marker='o', ecolor='#666666', linestyle='none')

H_f,_ = np.histogram(final_r[~did_disrupt], bins=bins)
axes[1].plot(bin_cen, H_f/V, ls='--', marker=None)

axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].set_xlim(1E-1, 1E2)
axes[1].set_ylim(1E-7, 1E2)
axes[1].set_xlabel(r"$r$ [kpc]")
axes[1].set_ylabel('GC density [kpc$^{-3}$]')

fig.tight_layout()

# fig.savefig("/Users/adrian/projects/uncluster/plots/GC-mass-radius.pdf")

---

# Ignore below here

---

### For a given mass-loss history and radius, estimate the length of the stream

$$
\begin{align}
l &= 2\Delta \theta\\
\Delta \theta_i &= \Delta\Omega_i \, t\\
\Delta\Omega_i &\approx \Omega_i \, \frac{\delta E}{E}\\
\delta E &= r_{\rm tid} \, \frac{d\Phi}{dr}\\
r_{\rm tid} &\approx \left(\frac{M_{\rm GC}}{3M(<r)}\right)^{1/3} \, r
\end{align}
$$
$\epsilon$ is the energy scale, $r_{\rm tid}$ is the tidal radius, $M(<r)$ is the enclosed mass of the Galaxy at radius $r$.

In [ ]:
def delta_Omega(r, M, pot):
    vc = circ_vel(r, pot)
    Omega = vc / r
    
    q = np.zeros(3)*r.unit
    q[0] = r
    
    # p = np.zeros(3)*vc.unit
    # p[0] = vc
    
    # r_tid = (M / (3*pot.mass_enclosed(q)))**(1/3.) * r
    # dE = r_tid * pot.gradient(q)[0]
    # E = pot.total_energy(q, p)
    dE_E = (M / (3*pot.mass_enclosed(q)))**(1/3.)
    
    return (dE_E*Omega)[0]

def get_r_tid(r, M, pot):
    r = np.atleast_1d(r)
    M = np.atleast_1d(M)
    
    q = np.zeros((3,r.size))*r.unit
    q[0] = r
    return (M / (3*pot.mass_enclosed(q)))**(1/3.) * r

In [ ]:
l = (delta_Omega(gc_r[0], gc_mass[0], pot) * (t_evolve-t_disrupt[0]))\
    .to(u.degree, equivalencies=u.dimensionless_angles())
l

TODO: As a really crude initial estimate (assuming spherical bg), if I know how much mass is lost at each time-step I can then allow it to linearly evolve in angle with a Gaussian cylinder radius set by the tidal radius. From this I can get the surface density. For example, the mass-loss info would give me a "strand" of stars, then I would convolve with a Gaussian to get a "stream"

In [ ]:
def kernel(r_tid, w):
    K = np.eye(3)
    K[0,0] = w.to(u.kpc).value
    K[1,1] = w.to(u.kpc).value
    K[2,2] = r_tid.to(u.kpc).value
    return K

def get_grid(r_tid, z_theta, dd=None):
    r_tid = r_tid.to(u.kpc).value
    z_theta = z_theta.to(u.kpc).value
    
    max_r_tid = np.nanmax(r_tid)
    max_z_theta = np.nanmax(z_theta)
    
    if dd is None:
        dd = max_r_tid.max()/4.
    
    print(np.arange(-3*max_r_tid, 3*max_r_tid+dd, dd).size, 
          np.arange(0., max_z_theta+max_r_tid + dd, dd).size)
    return
    dens_grid = np.meshgrid((np.arange(-5*r_tid, 5*r_tid+dd, dd),
                             np.arange(-5*r_tid, 5*r_tid+dd, dd),
                             np.arange(0., z_theta.max()+r_tid + dd, dd)))
    return dens_grid

In [ ]:
with h5py.File("../output/gc-evolve.h5", "r") as f:
    time = f['time'][:] * u.Unit(f['time'].attrs['unit'])
    t_evolve = time.max()
#     t_evolve = 2.5*u.Gyr

    for i in range(N_gcs): #[2455:2456]:
        M_i = f['mass/{}'.format(i)][:] * u.Unit(f['mass'].attrs['unit'])
        r_i = f['radius/{}'.format(i)][:] * u.Unit(f['radius'].attrs['unit'])
        
        dM = M_i[1:] - M_i[:-1]
        mean_M = (M_i[:-1] + M_i[1:])/2.
        mean_r = (r_i[:-1] + r_i[1:])/2.
        
        theta = np.nan*np.zeros(len(dM))*u.degree
        for j in range(len(dM)):
            dt = t_evolve-time[j]
            if dt < 0.:
                break
            
            theta[j] = (delta_Omega(mean_r[j], mean_M[j], pot) * dt)\
                       .to(u.degree, equivalencies=u.dimensionless_angles())
            
            if M_i[j] < 0.:
                break
            
        # turn angular coordinate into cartesian
        z_theta = (mean_r * theta).to(u.kpc, equivalencies=u.dimensionless_angles())

#         r_tid = get_r_tid(mean_r, mean_M, pot)
#         get_grid(r_tid, z_theta)
        # need to estimate mass density from this -- TODO: need width as a function of theta!
        
        #idx = np.isfinite(Omega)
        #H,edges = np.histogram(Omega[idx], bins=np.linspace(0.,180,180), weights=-dM[idx])

        break

In [ ]:
idx = np.isfinite(theta)
H,edges = np.histogram(theta[idx], bins=np.linspace(0.,420,64), weights=-dM[idx])
# H,edges = np.histogram(z_theta[idx], bins=np.linspace(0.,15,64), weights=-dM[idx])

pl.plot(edges[1:], H)

In [ ]:
print("hi")